In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

In [7]:
pat_df=pd.read_csv('../input/PATIENTS.csv',low_memory=False)
pat_df.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [8]:
pres_df=pd.read_csv('../input/PRESCRIPTIONS.csv',low_memory=False)
pres_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [9]:
note_df=pd.read_csv('../input/NOTEEVENTS.csv',low_memory=False)
note_df.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [10]:
print("Total no. of rows in this csv:{}".format(note_df.shape[0]))
print("Different types of category in this csv are:\n{}".format(note_df.CATEGORY.value_counts()))
print("Different types of description in this csv are:\n{}".format(note_df.DESCRIPTION.value_counts()))
print("Unique patient ids and their number of arrivals to the hospital are:\n{}".format(note_df.SUBJECT_ID.value_counts()))

Total no. of rows in this csv:2083180
Different types of category in this csv are:
Nursing/other        822497
Radiology            522279
Nursing              223556
ECG                  209051
Physician            141624
Discharge summary     59652
Echo                  45794
Respiratory           31739
Nutrition              9418
General                8301
Rehab Services         5431
Social Work            2670
Case Management         967
Pharmacy                103
Consult                  98
Name: CATEGORY, dtype: int64
Different types of description in this csv are:
Report                                                      1132519
Nursing Progress Note                                        191836
CHEST (PORTABLE AP)                                          169270
Physician Resident Progress Note                              62698
CHEST (PA & LAT)                                              43158
CT HEAD W/O CONTRAST                                          34485
Respiratory 

In [11]:
print("Number of unique patients:{}".format(len(note_df.SUBJECT_ID.unique())))
print("Number of patients who have visited the hospital only once:{}".format(note_df.shape[0]-len(note_df[note_df.duplicated(['SUBJECT_ID'],keep=False)])))

Number of unique patients:46146
Number of patients who have visited the hospital only once:167


In [19]:
note_df.loc[note_df['SUBJECT_ID'] == 109].CATEGORY.value_counts()

Nursing              410
Physician            363
Radiology            220
ECG                  118
Nursing/other         83
Discharge summary     34
General               19
Echo                  13
Respiratory           13
Case Management        2
Pharmacy               2
Nutrition              1
Social Work            1
Name: CATEGORY, dtype: int64

In [30]:
search_for=["Nursing", "Physician", "Radiology", "Discharge"]
new_df = note_df[note_df.CATEGORY.str.contains('|'.join(search_for))]

In [31]:
new_df.loc[new_df['SUBJECT_ID'] == 109].CATEGORY.value_counts()
new_df.to_csv('cleaned_final.csv',index='False')

Nursing              410
Physician            363
Radiology            220
Nursing/other         83
Discharge summary     34
Name: CATEGORY, dtype: int64

In [ ]:
#word counts
note_df['word_count'] = note_df['TEXT'].apply(lambda x: len(str(x).split(" ")))
note_df['word_count'].describe()

In [ ]:
train_des = note_df['TEXT'].fillna("not available").values
#train_subid=req_df['SUBJECT_ID'].values


In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
#from nltk.stem import WordNetLemmatizer 
  
#lemmatizer = WordNetLemmatizer() 
stop_words = set(stopwords.words('english')) 

def data_clean(big_list):
    for i in range(len(big_list)):
        curr_string=big_list[i]
        curr_string=curr_string.lower()
        word_tokens = word_tokenize(curr_string)
        filtered_sentence = [] 
        for w in word_tokens: 
            if (w not in stop_words) and (w.isalpha()):
                #word=lemmatizer.lemmatize(w)
                filtered_sentence.append(w)
        big_list[i]=' '.join(filtered_sentence)
    return big_list

In [ ]:
train_des=data_clean(train_des)


In [ ]:
df=pd.DataFrame({'TEXT':train_des})
df['word_count'] = df['TEXT'].apply(lambda x: len(str(x).split(" ")))
df['word_count'].describe()

In [ ]:
df.to_csv('text_cleaned.csv',index=False)

In [ ]:
from keras.layers import Dense, Input, Dropout, Activation
from keras.layers import Conv2D,GlobalMaxPooling2D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Dense,merge,Lambda,Flatten
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
import keras.backend as K

dim=300
filter_size=32
ksize=5

#Model1
inp1 = Input(shape=(None,dim,1))
x1 = Conv2D(filter_size, kernel_size=(ksize,dim) ,padding='valid', kernel_initializer='normal')(inp1)
x2 = GlobalMaxPooling2D()(x1)

#Model2
inp2 = Input(shape=(None,dim,1))
y1 = Conv2D(filter_size, kernel_size=(ksize,dim) ,padding='valid', kernel_initializer='normal')(inp2)
y2 = GlobalMaxPooling2D()(y1)

#Model3
merged = concatenate([x2,y2])
out = Dense(10, activation="softmax")(merged)
model = Model(inputs=[inp1,inp2], outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#Checkpoint
filepath="weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]


In [ ]:
from nltk.tokenize import word_tokenize 

def data_list(big_list):
    data_list=[]
    for i in range(len(big_list)):
        curr_string=big_list[i]
        curr_string=curr_string.lower()
        word_tokens = word_tokenize(curr_string)
        data_list.append(word_tokens) 
    return data_list


from gensim.models import Word2Vec

# train model
new_list=data_list(train_des)
model = Word2Vec(new_list,size=50, min_count=1)
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(len(words))
# access vector for one word
print(model['admission'])
# save model
model.save('model-mimic.bin')
# load model
new_model = Word2Vec.load('model-mimic.bin')
#print(new_model)

In [ ]:
embedding = Word2Vec.load('model-mimic.bin')

def gen_vector(record):
    vector = np.zeroes(dim)
    record = record.split(' ')
    for word in record:
        vector += embedding[word]
    return vector
    

In [5]:
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D , 
from keras.layers import Bidirectional, GlobalAveragePooling1D,GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import matplotlib.pyplot as plt
from keras.layers import Dense,merge
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate


max_visits=1400
dim=50

#model1
inp1 = Input(shape=(None,dim))
x2 = Bidirectional(CuDNNGRU(64, return_sequences=True))(inp1)
x3 = GlobalAveragePooling1D()(x2)

#model2
inp2 = Input(shape=(None,dim))
z2 = Bidirectional(CuDNNGRU(64, return_sequences=True))(inp2)
z3 = GlobalAveragePooling1D()(z2)

merged = concatenate([x3,z3])
y1 = Dense(256, activation='relu')(merged)
out = Dense(10, activation="softmax")(y1)
model = Model(inputs=[inp1,inp2], outputs=out)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

#Checkpoint
filepath="weights2.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# Fit model on training data
#history=model.fit([train_X1,train_X2],train_y,validation_split=0.4, epochs=10, batch_size=64, callbacks=callbacks_list, verbose=1)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_8 (InputLayer)            (None, None, 50)     0                                            
__________________________________________________________________________________________________
input_9 (InputLayer)            (None, None, 50)     0                                            
__________________________________________________________________________________________________
bidirectional_8 (Bidirectional) (None, None, 128)    44544       input_8[0][0]                    
__________________________________________________________________________________________________
bidirectional_9 (Bidirectional) (None, None, 128)    44544       input_9[0][0]                    
__________________________________________________________________________________________________
global_ave